In [1]:
import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.layers import Lambda
from fastai.callbacks import *
import pytorch_ssim as ssim
from superres import *
from torchvision.models import vgg16_bn
import czifile
import shutil

In [2]:
path_combo_src_hr = Path('/DATA/WAMRI/salk/uri/combo_src/')
path_combo_src_lr = Path('/DATA/WAMRI/salk/uri/combo_lr_pair/')
path_combo_hr = Path('/DATA/WAMRI/salk/uri/combo_hr/')
path_combo_lr = Path('/DATA/WAMRI/salk/uri/combo_lr/')
path_test_lr = Path('/DATA/WAMRI/salk/uri/newimg/')

In [3]:
def pull_id(fn):
    return fn.split('#')[-1].split('.')[0]

pair_names_by_id = {pull_id(lrfn.name):lrfn for lrfn in path_combo_src_lr.glob('*.tif')}

def match_lr_fn(x):
    return pair_names_by_id[pull_id(x.name)]

def get_czi_shape_info(czi):
    shape = czi.shape
    axes = czi.axes
    axes_dict = {axis:idx for idx,axis in enumerate(czi.axes)}
    shape_dict = {axis:shape[axes_dict[axis]] for axis in czi.axes}
    return axes_dict, shape_dict


def build_index(axes, ix_select):
    idx = [ix_select.get(ax, 0) for ax in axes]
    return tuple(idx)

def process_tiff(fn):
    lr_match = match_lr_fn(fn)
    shutil.copy(fn, path_combo_hr/fn.name)
    shutil.copy(lr_match, path_combo_lr/fn.name)
    

def process_czi(proc_fn):
    with czifile.CziFile(proc_fn) as proc_czf:
        proc_axes, proc_shape = get_czi_shape_info(proc_czf)
        channels = proc_shape['C']
        depths = proc_shape['Z']
        x,y = proc_shape['X'], proc_shape['Y']
        data = proc_czf.asarray()
        for channel in range(channels):
            depth = depths // 2
            idx = build_index(proc_axes, {'C': channel, 'Z':depth, 'X':slice(0,x),'Y':slice(0,y)})
            img = data[idx].astype(np.float)
            save_proc_fn = path_combo_hr/f'{proc_fn.stem}_{channel:02d}_{depth:03d}.tif'
            img /= img.max()
            pimg = PIL.Image.fromarray(img)
            pimg.save(save_proc_fn)
            cur_size = pimg.size
            new_size = (cur_size[0]//4, cur_size[1]//4)
            pimg.resize(new_size, resample=PIL.Image.BICUBIC).save(path_combo_lr/save_proc_fn.name)
    
def process_src(fn):
    if fn.suffix == '.tif':
        process_tiff(fn)
        pass
    elif fn.suffix == '.czi':
        process_czi(fn)

fnames = list(path_combo_src_hr.iterdir())

In [4]:
# print(len(fnames))
# for fn in progress_bar(fnames):
#     process_src(fn)

In [5]:
def get_basename(x):
    return x.name

base_names = list(set([get_basename(x) for x in list(path_combo_lr.iterdir())]))
train_names, valid_names = random_split(0.15, base_names)
valid_names = list(valid_names[0])

def is_validation_basename(x):
    return x.name in valid_names

def lr_to_hr_fn(x):
    return path_combo_hr/x.name

In [6]:
src = (GrayImageItemList
       .from_folder(path_combo_lr, '*.tif', label_class=GrayImageItemList)
       .split_by_valid_func(is_validation_basename)
       .label_from_func(lr_to_hr_fn))

In [7]:
def charbonnier_loss(inputs, targets):
    eps = 1e-6;
    d = inputs - targets
    e = torch.sqrt(d**2 + eps)
    return e.mean()



In [8]:
loss = charbonnier_loss
scale=4
n_resblocks=128
n_feats=32
res_scale= 0.1
model = WDSR(scale, n_resblocks, n_feats, res_scale, n_colors_in=1)
model = nn.DataParallel(model) #.eval().cuda()

In [ ]:
sz_lr = 128
scale,bs = 4,16
sz_hr = sz_lr*scale

learn = build_learner(model, bs, sz_lr, sz_hr, src, loss=loss) #, callback_fns=LossMetrics)
learn.lr_find()
learn.recorder.plot()

In [ ]:
sz_lr = 128
scale,bs = 4,16
sz_hr = sz_lr*scale

epochs, lr = 10, 1e-3
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, save='combo_1.0', loss=loss)

In [ ]:
sz_lr = 128
scale,bs = 4,16
sz_hr = sz_lr*scale

epochs, lr = 10, 2e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, load='combo_1.0_best', save='combo_1.1', loss=loss)

In [ ]:
sz_lr = 506
scale,bs = 4,1
sz_hr = sz_lr*scale
data = get_data(src, bs, sz_lr, sz_hr)
learn = Learner(data, model, loss_func=F.mse_loss).load('combo_1.1')
preds, ys = learn.get_preds(DatasetType.Valid)
preds = torch.clamp(preds, 0.,1.)



In [ ]:
idx = 2
hr = Image(ys[idx])
hr

In [ ]:
sr = Image(preds[idx])
sr

In [ ]:
lr = learn.data.valid_ds[idx][0]
data = lr.data.permute([1,2,0]).repeat([1,1,3]).detach().cpu().numpy()
img = PIL.Image.fromarray((data * 255).astype(np.uint8))
data = pil2tensor(img.resize((2024,2024), resample=PIL.Image.BICUBIC), dtype=np.float32)
lr = Image(data[0:1,:,:])
lr

In [ ]:
img1, img2 = sr.data[None],hr.data[None]
ssim.ssim(img1,img2), psnr(img1,img2), F.mse_loss(img1,img2)

In [ ]:
img1, img2 = lr.data[None], hr.data[None]
ssim.ssim(img1,img2), psnr(img1,img2), F.mse_loss(img1,img2)